In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install -q transformers

     |████████████████████████████████| 4.7 MB 38.5 MB/s 
     |████████████████████████████████| 120 kB 72.2 MB/s 
     |████████████████████████████████| 6.6 MB 58.0 MB/s 


In [3]:
!pip install xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import *
from tensorflow import *
import tensorflow as tf

In [5]:
df = pd.read_csv('/content/sample_data/bangla_news_comments_12000.csv')

In [6]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [7]:
df.replace(['Very Negative','Very Positive'],['Negative','Positive'],inplace=True)

In [8]:
df.replace(['Negative','Positive','Neutral'],[0,1,2],inplace=True)

In [9]:
df = df.sample(frac=1)

In [10]:
train_sample = df[0:int(.8*len(df))]

In [11]:
test_sample = df[int(.8*len(df)):]

In [12]:
from transformers import BertModel, BertTokenizer,TFRobertaForSequenceClassification,TFAutoModelForSequenceClassification,RobertaTokenizer,AutoTokenizer, AutoModel

In [13]:
PRE_TRAINED_MODEL_NAME = 'csebuetnlp/banglabert_generator'

In [ ]:
roberta = TFRobertaForSequenceClassification.from_pretrained(PRE_TRAINED_MODEL_NAME, num_labels=3, problem_type="multi_label_classification",from_pt=True)
tokenizer =BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [16]:
train_token = tokenizer.batch_encode_plus(train_sample['Comment'].tolist(), padding='max_length', max_length=128, truncation=True)
test_token = tokenizer.batch_encode_plus(test_sample['Comment'].tolist(),padding='max_length', max_length=128, truncation=True)

In [17]:
loss = tf.losses.SparseCategoricalCrossentropy()
optim = tf.optimizers.Adam(learning_rate=2e-5)
roberta.compile(loss=loss, optimizer=optim, metrics=['accuracy'])

In [18]:
train_temp = [np.asarray(train_token.input_ids), np.asarray(train_token.attention_mask)]

In [19]:
test_temp = [np.asarray(test_token.input_ids), np.asarray(test_token.attention_mask)]

In [20]:
roberta.fit(train_temp, train_sample['Tag'].to_numpy(), batch_size=32, epochs=1)

346/346 [==============================] - 85s 179ms/step - loss: 1.0544 - accuracy: 0.5115


In [21]:
roberta.predict(test_temp, batch_size=32).logits

array([[0.7418028 , 0.3739515 , 0.29508054],
       [0.7459044 , 0.3680386 , 0.30310088],
       [0.7328873 , 0.37621817, 0.3130639 ],
       ...,
       [0.7548317 , 0.3617491 , 0.31012   ],
       [0.745376  , 0.3762072 , 0.2963671 ],
       [0.73400635, 0.37582928, 0.3121039 ]], dtype=float32)